In [1]:
%pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

  Cloning https://github.com/sberbank-ai/Real-ESRGAN.git to /tmp/pip-req-build-gye0hmya
  Running command git clone --filter=blob:none --quiet https://github.com/sberbank-ai/Real-ESRGAN.git /tmp/pip-req-build-gye0hmya
  Resolved https://github.com/sberbank-ai/Real-ESRGAN.git to commit 362a0316878f41dbdfbb23657b450c3353de5acf
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusol

In [12]:
from RealESRGAN import RealESRGAN #type:ignore
from PIL import Image
import numpy as np
import torch
import cv2 as cv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "4"

model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')

device: cuda


In [17]:
def upScaling(video,width,height,fps,scale,device='cuda'):
    #creating output video after upscaling
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter("./output_video.mp4", fourcc, fps, (width * scale, height * scale))

    frame_count = 0
    while True:
        ret, frame = video.read()   # getting each frame
        if not ret:
            break

        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)  #converting to RGB


        sr_frame = model.predict(frame_rgb)   # upScaling using RealESRGAN

        #converting to the numpy array
        if isinstance(sr_frame, torch.Tensor):
            sr_frame = sr_frame.cpu().numpy()
        elif isinstance(sr_frame, np.ndarray):
            pass
        elif isinstance(sr_frame, Image.Image):
            sr_frame = np.array(sr_frame)
        else:
            raise ValueError("Unexpected output type from model.predict()")

        #converting upscale frame into BGR
        sr_frame_bgr = cv.cvtColor(sr_frame, cv.COLOR_RGB2BGR)

        #combining each frame into video
        out.write(sr_frame_bgr)

        frame_count += 1



    video.release()
    out.release()
    print("Upscaling complete.")

In [18]:
video = cv.VideoCapture("input2.mp4")
fps = video.get(cv.CAP_PROP_FPS)
width = int(video.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv.CAP_PROP_FRAME_HEIGHT))

In [19]:
print(f'Fps is: {fps} , Width is: {width}, Height is: {height}')

Fps is: 30.0 , Width is: 852, Height is: 480


In [20]:
upScaling(video,width,height,fps,4,device='cuda')

Upscaling complete.
